# Edit Data
Create and edit a **Record** object and write your changes to the MI server.

This notebook describes how to edit text-based attributes. See the following notebooks for additional attribute types:

- [Point and range attributes](./12_Add_Point_Range_Data.ipynb)
- [Date, integer, and logical attributes](./13_Add_Date_Integer_and_Logical_Data.ipynb)
- [Functional attributes](./07_Import_Functional_Data.ipynb)
- [Tabular attributes](./09_Edit_Tabular_Data.ipynb)
- [File, picture, and hyperlink attributes](./11_Add_Files_Pictures_and_Hyperlinks.ipynb)
- [Pseudo-attributes](./10_Edit_Pseudo-attributes.ipynb)

## Connect to MI
Get a database and table.

In [1]:
from datetime import datetime
from GRANTA_MIScriptingToolkit import granta as mpy

mi = mpy.connect('http://localhost/mi_servicelayer', autologon=True)
db = mi.get_db(db_key='MI_Training')
db.set_unit_system(unit_system='SI (Consistent)', absolute_temperatures=True)
tab = db.get_table('Design Data')

## Create a new record and path
Create a path in the table from a starting folder, in this case, the top level folder, using `path_from()`.
Specify `end_node` to create a new **Record** object at the end of the path with that name.

In [2]:
now = datetime.now().strftime("%c")
record_name = 'STK Example 5: {}'.format(now)
record = tab.path_from(None, tree_path=['High Alloy Steel', 'AMS 6520'], end_node=record_name)
record, record.parent

(<Record long name:STK Example 5: Tue May 10 22:37:06 2022>,
 <Record long name:AMS 6520>)

## Edit the record's attributes
Get some attributes associated with the record.

In [3]:
tab.bulk_fetch([record], attributes=['Common Name',
                                     'Product Form',
                                     'Statistical Basis',
                                     'Available mechanical properties'])
common_name = record.attributes['Common Name']
product_form = record.attributes['Product Form']
statistical_basis = record.attributes['Statistical Basis']
available_properties = record.attributes['Available mechanical properties']

Edit their data values.

In [4]:
common_name.value = 'STK Example 5 Test Material'
product_form.value = 'Plate'

*Statistical Basis* is a single valued discrete attribute, and can be set either with a string or a list containing a
single string.

In [5]:
print("{} supports multivalued data? {}".format(statistical_basis.name, statistical_basis.is_multivalued))
statistical_basis.value = 'S basis'

Statistical Basis supports multivalued data? False


*Available mechanical properties* is a multivalued discrete attribute. It can be set either with a string or a list of
strings for each value.

In [6]:
print("{} supports multivalued data? {}".format(available_properties.name, available_properties.is_multivalued))
available_properties.value = ['Tensile', 'Compression']

Available mechanical properties supports multivalued data? True


## Set the record's release status
If the record is being created in a version-controlled table,
decide whether the record should be released after creation.
The record is created in the **unreleased** state
to allow more data to be added later without creating a new version.

In [7]:
record.flag_for_release = False

## Write your changes to MI
First, specify the attributes on the record which you want to update on the server.

In [8]:
record.set_attributes([common_name, product_form, statistical_basis, available_properties])

Then write the changes to MI. The list of updated **Record** objects is returned.

In [9]:
record = mi.update([record])[0]
print('Record Name: {0}, State: {1}'.format(record.name, record.release_state))

Record Name: STK Example 5: Tue May 10 22:37:06 2022, State: Unreleased


## 'Not Applicable' flag
Further edits can be made to the same **Record** object. In this case, the *Condition* attribute is not relevant,
so the `is_applicable` property is set to False. This sets the attribute to 'Not Applicable' in Granta MI.

In [10]:
condition = record.attributes['Condition']
condition.is_applicable = False

Set the new attributes to update, and update again to write the latest changes to the server.
No further changes are required for this record, so the record will be released.

In [11]:
record.set_attributes([condition])
record.flag_for_release = True
record = mi.update([record])[0]

print('Record Name: {0}, State: {1}'.format(record.name, record.release_state))

Record Name: STK Example 5: Tue May 10 22:37:06 2022, State: Released


Check the `Record.all_versions` property for a dictionary of all versions of the record. The dictionary
contains a single version, confirming that only a single version was created in Granta MI.

In [12]:
record.all_versions

{'v1': <Record long name:STK Example 5: Tue May 10 22:37:06 2022>}

## Output the record's attributes
The attribute values are accessed via the same properties used for assignment.
*Statistical Basis* and *Available mechanical properties* are discrete attributes, so their `value` properties contain a
list of strings.
*Condition* is set to 'Not Applicable'; this is checked when printing the attribute value.

In [13]:
common_name = record.attributes['Common Name']
print('Common Name: {0}'.format(common_name.value))

product_form = record.attributes['Product Form']
print('Product Form: {0}'.format(product_form.value))

statistical_basis = record.attributes['Statistical Basis']
print('Statistical Basis: {0}'.format(statistical_basis.value[0]))

available_properties = record.attributes['Available mechanical properties']
result = ", ".join([prop for prop in available_properties.value])
print('Available mechanical properties: {0}'.format(result))

condition = record.attributes['Condition']
print('Condition: {0}'.format(condition.value if condition.is_applicable else '<Not Applicable>'))

Common Name: STK Example 5 Test Material
Product Form: Plate
Statistical Basis: S
Available mechanical properties: Tensile, Compression
Condition: <Not Applicable>
